# Model epic 5

Deze notebook implementeert een hybrid recommender systeem met content-based en collaborative filtering. Data wordt geladen vanuit een materialized view en ondergaat eerst content-based filtering op basis van tekstkenmerken van campagnes. Daarna wordt collaborative filtering toegepast met het Alternating Least Squares (ALS) model. De hybrid aanpak combineert de resultaten van beide methoden om een lijst van top 20 gebruikers te genereren die mogelijk geïnteresseerd zijn in een specifieke campagne. 

Het ALS model werd gekozen omdat het beter omkan met sparse matrices en impliciete ratings dan meer conventionele modellen zoals SVD. Met de content-based filtering wordt er getracht om *cold start* problemen te vermijden.

De itembased.ipynb notebook implementeert SVD en wordt niet verder gebruikt.

## 1. Imports

In [1]:
import os
import sqlalchemy
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from collections import Counter
from scipy.sparse import coo_matrix
from sqlalchemy import create_engine, text
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight
from sentence_transformers import SentenceTransformer
from implicit.evaluation import train_test_split, precision_at_k, AUC_at_k

c:\Users\hanne\OneDrive\Openbaar\Documenten\TI(schoolmap)\Data Engineering Project II\DEP2-GROEP-1\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Data inladen

In [2]:
load_dotenv()
DB_URL = os.getenv("DB_URL")
engine = create_engine(DB_URL)

try:
    connection = engine.connect()
    print("Successfully connected to the database")
except Exception as e:
    print(f"Failed to connect to the database: {e}")

print(f"SQLAlchemy version: {sqlalchemy.__version__}")

Successfully connected to the database
SQLAlchemy version: 2.0.21


In [3]:
# materialized view
query = text('SELECT * FROM epic_5')

try:
    df = pd.read_sql_query(query, connection)
except Exception as e:
    print(f"Failed to execute query: {e}")

df.head()

,PersoonId,CampagneId,aantal_sessies,aantal_bezoeken,SessieThema,SoortCampagne,TypeCampagne,ThemaDuurzaamheid,ThemaFinancieelFiscaal,ThemaInnovatie,ThemaInternationaalOndernemen,ThemaMobiliteit,ThemaOmgeving,ThemaSalesMarketingCommunicatie,ThemaStrategieEnAlgemeenManagement,ThemaTalent,ThemaWelzijn
0,097E0B38-AB67-E111-A00F-00505680000A,777D72E7-0591-E911-80FF-001DD8B72B62,1,0,Marketing & Sales,Offline,Opleiding,0,0,0,0,0,0,0,0,0,0
1,69C47E9C-A012-EA11-8108-001DD8B72B61,57A029E5-7A80-ED11-81AD-6045BD895B5A,6,0,Netwerking,Offline,Netwerkevenement,0,0,0,0,0,0,0,0,0,0
2,91203AB5-2253-EA11-810C-001DD8B72B62,1F0B5C31-79C3-EB11-8120-001DD8B72B61,1,0,Starten,Offline,Project,0,0,0,0,0,0,0,0,0,0
3,78CCAFF0-617A-E611-80DE-001DD8B72B61,49E02605-AD7A-E811-80F3-001DD8B72B61,1,0,Marketing & Sales,Offline,Opleiding,0,0,0,0,0,0,0,0,0,0
4,D5D81C7F-DF67-E111-A00F-00505680000A,543D1B29-11B0-E811-80F4-001DD8B72B62,1,0,Netwerking,Offline,Netwerkevenement,1,1,1,1,1,1,1,1,1,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52567 entries, 0 to 52566
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   PersoonId                           52567 non-null  object
 1   CampagneId                          52567 non-null  object
 2   aantal_sessies                      52567 non-null  int64 
 3   aantal_bezoeken                     52567 non-null  int64 
 4   SessieThema                         52567 non-null  object
 5   SoortCampagne                       52567 non-null  object
 6   TypeCampagne                        52567 non-null  object
 7   ThemaDuurzaamheid                   52567 non-null  int64 
 8   ThemaFinancieelFiscaal              52567 non-null  int64 
 9   ThemaInnovatie                      52567 non-null  int64 
 10  ThemaInternationaalOndernemen       52567 non-null  int64 
 11  ThemaMobiliteit                     52567 non-null  in

## 3. Content-based filtering

In [5]:
# tekstkenmerken van campagne sessies verzamelen
df_keywords = pd.DataFrame()
df_keywords['CampagneSessieInformatie'] = df['SessieThema'] + ' ' + df['SoortCampagne'] + ' ' + df['TypeCampagne']
df_keywords.head()

,CampagneSessieInformatie
0,Marketing & Sales Offline Opleiding
1,Netwerking Offline Netwerkevenement
2,Starten Offline Project
3,Marketing & Sales Offline Opleiding
4,Netwerking Offline Netwerkevenement


In [6]:
# sentence embedding
model_keywords = SentenceTransformer('all-MiniLM-L6-v2')
keyword_vectors = model_keywords.encode(df_keywords['CampagneSessieInformatie'].tolist())

In [7]:
# matrix die toont goed campagne sessies op elkaar lijken op basis van de tekstkenmerken
similarity_matrix = np.dot(keyword_vectors, keyword_vectors.T)
print(similarity_matrix)

[[1.0000002  0.41175914 0.44478056 ... 0.33685726 0.32816333 0.41175914]
 [0.41175914 1.0000001  0.5295649  ... 0.762505   0.4511339  1.0000001 ]
 [0.44478056 0.5295649  0.9999999  ... 0.48442447 0.43705958 0.5295649 ]
 ...
 [0.33685726 0.762505   0.48442447 ... 0.99999994 0.7873782  0.762505  ]
 [0.32816333 0.4511339  0.43705958 ... 0.7873782  1.         0.4511339 ]
 [0.41175914 1.0000001  0.5295649  ... 0.762505   0.4511339  1.0000001 ]]


## 4. Collaborative filtering

SOURCE: https://benfred.github.io/implicit/index.html (Alternating Least Squares)

In [8]:
# toch groeperen op aantal sessies per persoon per campagne, geen scheiding tussen sessiethema
df = df.groupby(['PersoonId', 'CampagneId'])['aantal_sessies'].sum().reset_index()

# persoonid en campagneid omzetten naar integers voor de sparse matrix invoer van het model
persoon_mapping = {id: i for i, id in enumerate(df['PersoonId'].unique())}
campagne_mapping = {id: i for i, id in enumerate(df['CampagneId'].unique())}

# reverse mapping om later de integer terug naar de string te krijgen
persoon_reverse_mapping = {i: id for id, i in persoon_mapping.items()}
campagne_reverse_mapping = {i: id for id, i in campagne_mapping.items()}

# mapping toepassen op dataframe
df['PersoonId'] = df['PersoonId'].map(persoon_mapping)
df['CampagneId'] = df['CampagneId'].map(campagne_mapping)

In [9]:
df.head()

,PersoonId,CampagneId,aantal_sessies
0,0,0,1
1,0,1,1
2,1,2,1
3,2,3,1
4,3,4,1


In [10]:
# sparse matrix maken met aantal sessies als waarden
campagne_persoon_sessies = coo_matrix((df['aantal_sessies'], (df['CampagneId'], df['PersoonId'])))

In [11]:
# matrix wegen, om hoge aantal sessies minder zwaar te laten wegen
campagne_persoon_sessies = bm25_weight(campagne_persoon_sessies, K1=100, B=0.5)

# matrix transponeren omdat implicit werkt met (user, item) in plaats van (item, user)
persoon_sessies = campagne_persoon_sessies.T.tocsr()

In [12]:
# trainset en testset maken via de implicit library
trainset, testset = train_test_split(persoon_sessies, train_percentage=0.8, random_state=42)

In [13]:
# nodig voor de ALS berekening
os.environ["OPENBLAS_NUM_THREADS"] = "1"

In [14]:
# ALS model trainen
model = AlternatingLeastSquares(factors=128, regularization=0.15, alpha=5.0, iterations=100, random_state=42)
model.fit(trainset)

c:\Users\hanne\OneDrive\Openbaar\Documenten\TI(schoolmap)\Data Engineering Project II\DEP2-GROEP-1\venv\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 100/100 [00:47<00:00,  2.13it/s]


In [15]:
# ALS model evalueren
precision = precision_at_k(model, trainset, testset, K=10, num_threads=4)
auc = a = AUC_at_k(model, trainset, testset, K=10, num_threads=4)

print(f"Precision: {precision}", f"AUC: {auc}", sep="\n")

100%|██████████| 6023/6023 [00:00<00:00, 11160.29it/s]

Precision: 0.29319591965506364
AUC: 0.6329666986372277


## 5. ALS model functies

In [16]:
# geef gelijkaardige campagnes
def get_similar_items(item_id):
    similar = model.similar_items(item_id, 10+1) 
    return similar[0][1:]

# get_similar_items(5)

In [17]:
# geef gelijkaardige personen
def get_similar_users(user_id):
    similar = model.similar_users(user_id, 10+1) 
    return similar[0][1:]
    
# get_similar_users(5)

In [18]:
# geef recommendaties
def ALS_recommend(user_id):
    item_ids, scores = model.recommend(user_id, persoon_sessies[user_id], N=20, filter_already_liked_items=False)
    return item_ids

# ALS_recommend(5)

## 6. Hybrid model

Deze functie geeft een lijst van gebruikers terug die mogelijk geïnteresseerd zijn in een specifieke campagne. Het baseert de interesse van gebruikers op de gelijkenis tussen de opgegeven campagne en andere campagnes, evenals de aanbevelingen van het ALS-model voor die gebruikers.

Parameters:
- campaign_id (str): De unieke identificatie van de campagne waarvoor we geïnteresseerde gebruikers willen vinden.

Returns:
- top_user_ids (list): Een lijst van de top 20 gebruikers die mogelijk geïnteresseerd zijn in de opgegeven campagne.

In [19]:
def get_interested_users(campaign_id):
    
    # map de campagne van string naar integer
    campaign_index = campagne_mapping[campaign_id]
    
    # gelijkenis scores uit de similarity matrix van de campagne met andere campagnes
    similarity_scores = list(enumerate(similarity_matrix[campaign_index]))
    # sorteer de scores van hoog naar laag
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # neem de top 5 scores
    similar_campaign_indices = [i for i, _ in similarity_scores[:5]]

    interested_users = []

    # voor elke van de top 5 gelijkaardige campagnes, neem de personen die het meest geinteresseerd zijn
    for similar_campaign_index in similar_campaign_indices:
        for user_index in persoon_mapping.values():
            # verkrijg de recommended campagnes voor deze persoon
            recommended_items = ALS_recommend(user_index)
            # als de campagne in de recommended campagnes zit, voeg deze persoon toe aan de lijst
            if similar_campaign_index in recommended_items:
                interested_users.append(user_index)
    
    # tel de frequentie van de personen
    user_counter = Counter(interested_users)

    # geef de 20 meest voorkomende personen
    top_user_indices = [user_index for user_index, _ in user_counter.most_common(20)]

    # map de personen van integer naar string
    top_user_ids = [persoon_reverse_mapping[user_index] for user_index in top_user_indices]

    return top_user_ids

print(get_interested_users('8FCA1D31-1EB7-E811-80F4-001DD8B72B62'))

['0368D4F4-EDF3-E411-90DE-005056B06EC4', '06792FBD-9CA8-E311-AC5C-005056B06EB4', '0E41BD97-5B69-E111-B43A-00505680000A', '13DBB079-A255-E211-9B51-005056B06EC4', '151A3AF4-054D-E711-80E7-001DD8B72B61', '15903CC0-4F69-E111-B43A-00505680000A', '1934C230-A497-E711-80EB-001DD8B72B62', '1F3CBF15-2368-E111-A00F-00505680000A', '2AE504CE-6E13-E211-9DAA-005056B06EB4', '2CDEF901-FC08-E911-80FA-001DD8B72B61', '2D4B2AE6-D051-E811-80F0-001DD8B72B62', '2D610064-37EE-E811-80FA-001DD8B72B62', '30A2D1FF-46F4-E211-A85C-005056B06EC4', '3248EEF0-6AA4-E311-AC5C-005056B06EB4', '327CB406-D171-E211-A85C-005056B06EC4', '3603477F-9B3B-E711-80E6-001DD8B72B61', '38A08B10-5169-E111-B43A-00505680000A', '3A75398C-9955-E711-80E7-001DD8B72B61', '3D14D46D-FD67-E111-A00F-00505680000A', '41A7D637-5D13-E911-80FA-001DD8B72B61']
